In [1]:
import pandas as pd
from tqdm.auto import tqdm

In [2]:
import pandas as pd
import rdkit
from rdkit import Chem
from rdkit.Chem.rdchem import HybridizationType
from rdkit.Chem.rdchem import BondType as BT
from rdkit.Chem import AllChem
import torch
import numpy as np
from tqdm import tqdm

In [ ]:
df = pd.read_csv('data.csv')

In [19]:
ATOM_LIST = list(range(1,119))
CHIRALITY_LIST = [
    Chem.rdchem.ChiralType.CHI_UNSPECIFIED,
    Chem.rdchem.ChiralType.CHI_TETRAHEDRAL_CW,
    Chem.rdchem.ChiralType.CHI_TETRAHEDRAL_CCW,
    Chem.rdchem.ChiralType.CHI_OTHER
]
BOND_LIST = [
    BT.SINGLE,
    BT.DOUBLE,
    BT.TRIPLE,
    BT.AROMATIC
]
BONDDIR_LIST = [
    Chem.rdchem.BondDir.NONE,
    Chem.rdchem.BondDir.ENDUPRIGHT,
    Chem.rdchem.BondDir.ENDDOWNRIGHT
]
BONDTYPES_LIST = [Chem.rdchem.BondType.names,
                  Chem.rdchem.BondType.values
                  ]

def get_graph_columns(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return torch.tensor([[], []], dtype=torch.long), torch.tensor(np.array([]), dtype=torch.long)

    N = mol.GetNumAtoms()
    M = mol.GetNumBonds()

    type_idx = []
    chirality_idx = []
    atomic_number = []
    
    for atom in mol.GetAtoms():
        type_idx.append(ATOM_LIST.index(atom.GetAtomicNum()))
        chirality_idx.append(CHIRALITY_LIST.index(atom.GetChiralTag()))
        atomic_number.append(atom.GetAtomicNum())
    
    x1 = torch.tensor(type_idx, dtype=torch.long).view(-1,1)
    x2 = torch.tensor(chirality_idx, dtype=torch.long).view(-1,1)
    x = torch.cat([x1, x2], dim=-1)

    row, col, edge_feat = [], [], []
    for bond in mol.GetBonds():
        start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        row += [start, end]
        col += [end, start]
        # edge_type += 2 * [MOL_BONDS[bond.GetBondType()]]
        edge_feat.append([
            bond.GetBondTypeAsDouble(),
            BONDDIR_LIST.index(bond.GetBondDir())
        ])
        edge_feat.append([
            bond.GetBondTypeAsDouble(),
            BONDDIR_LIST.index(bond.GetBondDir())
        ])

    edge_index = torch.tensor([row, col], dtype=torch.long)
    edge_attr = torch.tensor(edge_feat, dtype=torch.long)

    return [x, edge_index, edge_attr]

In [38]:
tqdm.pandas()
d = df['Smiles'].iloc[:100].progress_apply(get_graph_columns)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 1387.55it/s]


In [39]:
x = []
edge_idx = []
edge_attr = []

In [40]:
[x.append(i[0].tolist()) for i in d]
[edge_idx.append(i[1].tolist()) for i in d]
[edge_attr.append(i[2].tolist()) for i in d]
d = []

In [25]:
df['Smiles'].iloc[:100].progress_apply(get_graph_columns)[0]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 1529.66it/s]


3

In [30]:
df['x'] = x
df['edge_idx'] = edge_idx
df['edge_attr'] = edge_attr

ValueError: Length of values (100) does not match length of index (2372673)

In [50]:
import numpy as np

In [45]:
lens = []
for i in range(0, 100):
    lens.append(len(edge_idx[i][0]))

34

In [47]:
import torch
from torch_geometric.data import Data



data = Data(x=x, edge_index=edge_index, edge_attr = edge_attr)


In [49]:
data

Data(x=[100], edge_index=[2, 54], edge_attr=[100])